In [1]:
import os
current_directory = os.getcwd()
print("Current working directory:", current_directory)

Current working directory: /Users/sawad/dev/myapps/u-on-zu_data-processing


In [2]:
#出力先ディレクトリの設定
output_dir = './processed'

In [4]:
import pandas as pd
import numpy as np

# 作成した観測所一覧を読み込み
df_observ=pd.read_csv('./processed/observatories_finalize.csv', encoding='utf-8')

# 手動で入力した観測所種別データを読み込み
df_observ_type=pd.read_csv('./raw/観測所種別 - 観測所種別.csv', encoding='utf-8')
df_observ_type=df_observ_type[['観測所番号', '観測所種別']]

# 観測所種別で左結合
merged_df = pd.merge(df_observ, df_observ_type, on='観測所番号')

# 作成した各地点データを読み込み
df_climate=pd.read_csv('./processed/new_df.csv', encoding='utf-8')

# 「観測所名」で左結合
#観測所名は重複する場合があるので，重複のある場合は第2キーとして都府県振興局名で紐づける
merged_df = pd.merge(merged_df, df_climate, on=['観測所名', '都府県振興局'], how='left')




,都府県振興局,観測所番号,種類,観測所名,ｶﾀｶﾅ名,気象情報等に表記する名称,所在地,緯度(度),緯度(分),経度(度),経度(分),海面上の高さ(ｍ),風速計の高さ(ｍ),温度計の高さ(ｍ),観測開始年月日,備考1,備考2,latitude,longitude,観測所種別
0,宗谷,11001,四,宗谷岬,ｿｳﾔﾐｻｷ,稚内市宗谷岬,稚内市宗谷岬,45,31.2,141,56.1,26,10.1,1.5,昭53.10.30,－,－,45.520000,141.935000,2
1,宗谷,11016,官,稚内,ﾜｯｶﾅｲ,稚内市開運,稚内市開運 稚内地方気象台,45,24.9,141,40.7,3,24.1,－,#昭50.4.1,11903,－,45.415000,141.678333,1
2,宗谷,11046,四,礼文,ﾚﾌﾞﾝ,礼文町香深,礼文郡礼文町大字香深村トンナイ,45,18.3,141,2.7,65,9.9,1.5,平15.10.17,－,－,45.305000,141.045000,2
3,宗谷,11076,四,浜鬼志別,ﾊﾏｵﾆｼﾍﾞﾂ,猿払村浜鬼志別,宗谷郡猿払村浜鬼志別,45,20.1,142,10.2,13,9.9,1.5,昭53.10.30,11900,－,45.335000,142.170000,2
4,宗谷,11121,四,沼川,ﾇﾏｶﾜ,稚内市沼川,稚内市声問村沼川,45,14.9,141,51.1,23,10,1.5,昭52.10.20,11905,湿度を除く,45.248333,141.851667,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
891,沖縄,94017,官,与那国島,ﾖﾅｸﾞﾆｼﾞﾏ,与那国町祖納,八重山郡与那国町与那国 与那国島特別地域気象観測所,24,28.0,123,0.6,30,14.4,－,昭53.4.25,－,－,24.466667,123.010000,1
892,沖縄,94062,官,西表島,ｲﾘｵﾓﾃｼﾞﾏ,竹富町上原,八重山郡竹富町字上原宇那利崎 西表島特別地域気象観測所,24,25.6,123,45.9,10,10.1,－,平15.3.12,－,－,24.426667,123.765000,1
893,沖縄,94081,官,石垣島,ｲｼｶﾞｷｼﾞﾏ,石垣市登野城,石垣市登野城 石垣島地方気象台,24,20.2,124,9.8,6,28.9,－,昭52.3.8,－,－,24.336667,124.163333,1
894,沖縄,94101,四,大原,ｵｵﾊﾗ,竹富町大原,八重山郡竹富町南風見,24,15.9,123,52.3,33,10,1.5,昭53.3.14,－,－,24.265000,123.871667,2


In [5]:

# 気候データが空欄の都市を削除
merged_df.replace('', np.nan, inplace=True)   #一旦空欄をNanに変換
merged_df.dropna(subset=['temp_jan'], inplace=True)   # 1月の平均気温で判定する

# 出力
merged_df.to_csv(os.path.join(output_dir, 'merged_df.csv'), index=False)

In [7]:
# DB登録用jsonデータを作成

import json

# 空欄のある行を削除
df = merged_df.dropna()

# JSON変換の関数定義
def convert_row_to_db_json(row):
    id_value = str(row["観測所番号"])
    inner_area_id_value = id_value[:2]
    
    return {
        "id": row["観測所番号"],
        "country_id": 123,
        "inner_area_id": inner_area_id_value,
        "name": row["観測所名"],
        "observ_type": row["観測所種別"],
        "data": {
            "temp_ave": {
                "jan": row["temp_jan"],
                "feb": row["temp_feb"],
                "mar": row["temp_mar"],
                "apr": row["temp_apr"],
                "may": row["temp_may"],
                "jun": row["temp_jun"],
                "jul": row["temp_jul"],
                "aug": row["temp_aug"],
                "sep": row["temp_sep"],
                "oct": row["temp_oct"],
                "nov": row["temp_nov"],
                "dec": row["temp_dec"]
            },
            "rain": {
                "jan": row["rain_jan"],
                "feb": row["rain_feb"],
                "mar": row["rain_mar"],
                "apr": row["rain_apr"],
                "may": row["rain_may"],
                "jun": row["rain_jun"],
                "jul": row["rain_jul"],
                "aug": row["rain_aug"],
                "sep": row["rain_sep"],
                "oct": row["rain_oct"],
                "nov": row["rain_nov"],
                "dec": row["rain_dec"]
            },
            "position": {
                "lat": row["latitude"],
                "lng": row["longitude"],
                "alt": row["海面上の高さ(ｍ)"]
            }
        }
    }


# 各行を変換
db_json_data = [convert_row_to_db_json(row) for index, row in df.iterrows()]

# JSONを文字列に変換して表示
db_json_output = json.dumps(db_json_data, indent=2, ensure_ascii=False)
print(db_json_output)

# JSONをファイルに保存
output_filename = os.path.join(output_dir, "cities.json")
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(db_json_data, f, indent=2, ensure_ascii=False)

[
  {
    "id": 11001,
    "country_id": 123,
    "inner_area_id": "11",
    "name": "宗谷岬",
    "observ_type": 2,
    "data": {
      "temp_ave": {
        "jan": -4.5,
        "feb": -4.6,
        "mar": -1.1,
        "apr": 3.9,
        "may": 8.2,
        "jun": 11.9,
        "jul": 16.1,
        "aug": 18.7,
        "sep": 16.4,
        "oct": 10.8,
        "nov": 3.6,
        "dec": -2.1
      },
      "rain": {
        "jan": 23.5,
        "feb": 18.4,
        "mar": 21.5,
        "apr": 31.0,
        "may": 58.0,
        "jun": 58.8,
        "jul": 106.3,
        "aug": 126.5,
        "sep": 123.8,
        "oct": 114.6,
        "nov": 94.4,
        "dec": 54.0
      },
      "position": {
        "lat": 45.52,
        "lng": 141.935,
        "alt": 26
      }
    }
  },
  {
    "id": 11016,
    "country_id": 123,
    "inner_area_id": "11",
    "name": "稚内",
    "observ_type": 1,
    "data": {
      "temp_ave": {
        "jan": -4.3,
        "feb": -4.3,
        "mar": -0.6,
    

In [8]:
# フロント読み込み用マッピングデータの作成

# JSON変換の関数定義
def convert_row_to_front_json(row):
    id_value = str(row["観測所番号"])
    inner_area_id_value = id_value[:2]
    
    return {
        "id": row["観測所番号"],
        "country_id": 123,
        "inner_area_id": inner_area_id_value,
        "name": row["観測所名"],
        "observ_type": row["観測所種別"],
        "position": {
            "lat": row["latitude"],
            "lng": row["longitude"],
            "alt": row["海面上の高さ(ｍ)"]
            }
        }

# 各行を変換
front_json_data = [convert_row_to_front_json(row) for index, row in df.iterrows()]

# JSONを文字列に変換して表示
fron_json_output = json.dumps(front_json_data, indent=2, ensure_ascii=False)
print(fron_json_output)

# JSONをファイルに保存
output_filename = os.path.join(output_dir, "cityIdMapping.json")
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(front_json_data, f, indent=2, ensure_ascii=False)


[
  {
    "id": 11001,
    "country_id": 123,
    "inner_area_id": "11",
    "name": "宗谷岬",
    "observ_type": 2,
    "position": {
      "lat": 45.52,
      "lng": 141.935,
      "alt": 26
    }
  },
  {
    "id": 11016,
    "country_id": 123,
    "inner_area_id": "11",
    "name": "稚内",
    "observ_type": 1,
    "position": {
      "lat": 45.415,
      "lng": 141.67833333333334,
      "alt": 3
    }
  },
  {
    "id": 11046,
    "country_id": 123,
    "inner_area_id": "11",
    "name": "礼文",
    "observ_type": 2,
    "position": {
      "lat": 45.305,
      "lng": 141.045,
      "alt": 65
    }
  },
  {
    "id": 11076,
    "country_id": 123,
    "inner_area_id": "11",
    "name": "浜鬼志別",
    "observ_type": 2,
    "position": {
      "lat": 45.335,
      "lng": 142.17,
      "alt": 13
    }
  },
  {
    "id": 11121,
    "country_id": 123,
    "inner_area_id": "11",
    "name": "沼川",
    "observ_type": 2,
    "position": {
      "lat": 45.248333333333335,
      "lng": 141.851666666666